In [1]:
import datetime
import warnings
warnings.filterwarnings('ignore')
import torch
from glob import glob
import os
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from tqdm import tqdm
import numpy as np
from torchvision import transforms
import torchvision.models as models
import torch.nn as nn
from torch.nn import functional as F
from sklearn.model_selection import KFold
import time
#from efficientnet_pytorch import EfficientNet
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split


import matplotlib.pyplot as plt
#from torch_poly_lr_decay import PolynomialLRDecay
from torchvision import models
from sklearn.metrics import accuracy_score, log_loss
import math
import librosa

import random
import torchaudio

from tools.file_io import load_csv_file

os.getcwd()

'/home/clim-lab/바탕화면/main_drive/jiwon/retrieval'

In [4]:
from tools.audio_augment import RandomClip, RandomSpeedChange, RandomBackgroundNoise, ComposeTransform

In [2]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0,1'
torch.set_num_threads(8)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


In [7]:
import os
wav_path = os.listdir('./data/Clotho/waveforms/train/')

audio_ex1, sr1 = torchaudio.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]))
print(audio_ex1)
print(audio_ex1.shape)

audio_ex2, sr2 = librosa.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]))
print(audio_ex2)
print(audio_ex2.shape)

data_path = '/home/clim-lab/바탕화면/main_drive/jiwon/retrieval/data/Clotho'
#csv_path = os.path.join(data_path, 'csv_files/train.csv')
csv_path = os.path.join(data_path, 'csv_files/')

#csv_path = './data/Clotho/csv_files/'
trains_aug = os.path.join(csv_path, 'train_melt.csv')
vals_c = os.path.join(csv_path, 'val.csv')

train_csv_list = load_csv_file(os.path.join(csv_path, 'train_melt.csv'))
display(train_csv_list[:2])
display(pd.DataFrame(train_csv_list))

tensor([[ 0.0005,  0.0007,  0.0007,  ..., -0.0023, -0.0020, -0.0016]])
torch.Size([1, 1261260])
[ 0.00042238  0.00072047  0.00072089 ... -0.00280746 -0.00228519
 -0.00202156]
(630630,)


[OrderedDict([('file_name', ' Ambience Birds.wav'),
              ('caption',
               'A wild assortment of birds are chirping and calling out in nature.')]),
 OrderedDict([('file_name', ' Ambience Birds.wav'),
              ('caption',
               'Several different types of bird are tweeting and making calls.')])]

,file_name,caption
0,Ambience Birds.wav,A wild assortment of birds are chirping and ca...
1,Ambience Birds.wav,Several different types of bird are tweeting a...
2,Ambience Birds.wav,"Birds tweeting and chirping happily, engine in..."
3,Ambience Birds.wav,An assortment of wild birds are chirping and ...
4,Ambience Birds.wav,Birds are chirping and making loud bird noises.
...,...,...
19190,zippers01.wav,A jacket or bag is being zipped and unzipped.
19191,zippers01.wav,A long zipper is clasped frequently back and f...
19192,zippers01.wav,A zipper to a suitcase is being opened and clo...
19193,zippers01.wav,Long zipper being zipped back and forth multip...


In [22]:
torchaudio.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]))

(tensor([[ 0.0005,  0.0007,  0.0007,  ..., -0.0023, -0.0020, -0.0016]]), 44100)

In [23]:
librosa.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]))

(array([ 0.00042238,  0.00072047,  0.00072089, ..., -0.00280746,
        -0.00228519, -0.00202156], dtype=float32),
 22050)

In [27]:
a1, s1 = torchaudio.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]))
a1_trans = torchaudio.functional.resample(a1, orig_freq=44100, new_freq = 32000)
a1_trans.reshape(-1).numpy()
a2, s2 = librosa.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]), sr=32000)

display(a1_trans)
display(a1_trans.shape)
display(a2)
display(a2.shape)

tensor([[ 0.0004,  0.0007,  0.0007,  ..., -0.0023, -0.0022, -0.0018]])

torch.Size([1, 915200])

array([ 0.00042718,  0.00074288,  0.00065001, ..., -0.00224547,
       -0.00219583, -0.00178362], dtype=float32)

(915200,)

In [11]:
import time
from itertools import chain

import h5py
import numpy as np
import librosa
from re import sub
from loguru import logger
from pathlib import Path
from tqdm import tqdm
from tools.file_io import load_csv_file, write_pickle_file

def load_metadata(dataset, csv_file):
    """Load meta data of Clotho
    """
    if dataset == 'AudioCaps' and 'train' in csv_file:
        caption_field = None
    else:
        caption_field = ['caption_{}'.format(i) for i in range(1, 6)]
    csv_list = load_csv_file(csv_file)

    audio_names = []
    captions = []

    for i, item in enumerate(csv_list):

        audio_name = item['file_name']
        if caption_field is not None:
            item_captions = [_sentence_process(item[cap_ind], add_specials=False) for cap_ind in caption_field]
        else:
            item_captions = _sentence_process(item['caption'])
        audio_names.append(audio_name)
        captions.append(item_captions)

    meta_dict = {'audio_name': np.array(audio_names), 'captions': np.array(captions)}

    return meta_dict


def _create_vocabulary(captions):
    vocabulary = []
    for caption in captions:
        caption_words = caption.strip().split()
        vocabulary.extend(caption_words)
    words_list = list(set(vocabulary))
    words_list.sort(key=vocabulary.index)
    words_freq = [vocabulary.count(word) for word in words_list]
    words_list.append('<sos>')
    words_list.append('<eos>')
    words_list.append('<ukn>')
    words_freq.append(len(captions))
    words_freq.append(len(captions))
    words_freq.append(0)

    return words_list, words_freq


def _sentence_process(sentence, add_specials=False):

    # transform to lower case
    sentence = sentence.lower()

    if add_specials:
        sentence = '<sos> {} <eos>'.format(sentence)

    # remove any forgotten space before punctuation and double space
    sentence = sub(r'\s([,.!?;:"](?:\s|$))', r'\1', sentence).replace('  ', ' ')

    # remove punctuations
    sentence = sub('[,.!?;:\"]', ' ', sentence).replace('  ', ' ')
    return sentence


def pad_or_truncate(x, audio_length):
    """Pad all audio to specific length."""
    length = len(x)
    if length <= audio_length:
        return np.concatenate((x, np.zeros(audio_length - length)), axis=0), length
    else:
        return x[:audio_length], audio_length

    
#######
splits = ['train', 'val', 'test']
sampling_rate = 32000
audio_duration = 30
max_audio_length = audio_duration * sampling_rate # 30 * 32000
dataset='Clotho'

data_path = '/home/clim-lab/바탕화면/main_drive/jiwon/retrieval/data/Clotho'
csv_path = os.path.join(data_path, 'csv_files/')
#csv_path = os.path.join(data_path, 'csv_files/train.csv')

train_csv_path = os.path.join(csv_path, 'train.csv')


#train_data 만 load 시키기
train_csv_path = os.path.join(csv_path, 'train.csv')
train_audio_dir = os.path.join(data_path, 'waveforms/train')
train_hdf5_path = os.path.join(data_path, 'hdf5s/train/train.h5')

train_meta_dict = load_metadata(dataset, train_csv_path)
    # meta_dict: {'audio_names': [], 'captions': []}
print(train_meta_dict)

import pandas as pd
pd.read_csv(train_csv_path)

{'audio_name': array(['Distorted AM Radio noise.wav', 'Paper_Parchment_Rustling.wav',
       '03 Whales Slowing Down.wav', ...,
       'cold waterdrops in a hot pot.wav',
       'Dry leaves falling on cement floor.wav', 'Wood chips.wav'],
      dtype='<U133'), 'captions': array([['a muddled noise of broken channel of the tv',
        'a television blares the rhythm of a static tv ',
        'loud television static dips in and out of focus',
        'the loud buzz of static constantly changes pitch and volume ',
        'heavy static and the beginnings of a signal on a transistor radio'],
       ['a person is turning a map over and over ',
        'a person is very carefully rapping a gift for someone else ',
        'a person is very carefully wrapping a gift for someone else ',
        'he sighed as he turned the pages of the book stopping to scan the information ',
        'papers are being turned stopped then turned again and someone is breathing '],
       ['several barnyard animal

,file_name,caption_1,caption_2,caption_3,caption_4,caption_5
0,Distorted AM Radio noise.wav,A muddled noise of broken channel of the TV,A television blares the rhythm of a static TV.,Loud television static dips in and out of focus,The loud buzz of static constantly changes pit...,heavy static and the beginnings of a signal on...
1,Paper_Parchment_Rustling.wav,A person is turning a map over and over.,A person is very carefully rapping a gift for ...,A person is very carefully wrapping a gift for...,"He sighed as he turned the pages of the book, ...","papers are being turned, stopped, then turned ..."
2,03 Whales Slowing Down.wav,Several barnyard animals mooing in a barn whil...,"The vocalization of several whales, along with...","Underwater, large numbers of shrimp clicking a...",Whales sing to one another over the flowing wa...,wales sing to one another with water flowing i...
3,Rope tied to boat in port.wav,An office chair is squeaking as someone bends ...,Popping and squeaking gradually tapers off to ...,Someone is opening a creaky door slowly while ...,Squeaking and popping followed by gradual popp...,an office chair is squeaking as someone leans ...
4,carpenter bee.wav,A flying bee is buzzing loudly around an objec...,An annoying fly is buzzing loudly and consiste...,An insect buzzing in the foreground as birds c...,"An insect trapped in a spider web struggles, b...","Outdoors, insect trapped in a spider web and t..."
...,...,...,...,...,...,...
3834,Metallic Ping CPU Heatsink.wav,Tools are being tried to make different sounds.,The metallic clang is made at different and va...,One at a time the metal chimes are being chimed.,A metallic clang is made at various pitches.,Metal chimes being chimed one at a time.
3835,Fumbling.wav,"multiple items are picked up, and a tin pan is...",They are sorting through objects and dropping ...,A person works moving cans and other items and...,Cans and items are being moved around clunking...,Going through all of the trash can noisily.
3836,cold waterdrops in a hot pot.wav,Someone is flipping over food on a hot grill.,Someone is flipping over food above a hot grill.,All recorded audio is drowned out by the loudn...,Someone waters down a boat with a high pressur...,"A match is lit, then another match is lit."
3837,Dry leaves falling on cement floor.wav,Tapping noises are being made before paper is ...,Repeated tapping hits a hard surface and multi...,A person is typing and pauses a few times in b...,They are continually dropping clips on the table.,Sticks crunch and break while being walked on.


In [12]:
train_meta_dict

{'audio_name': array(['Distorted AM Radio noise.wav', 'Paper_Parchment_Rustling.wav',
        '03 Whales Slowing Down.wav', ...,
        'cold waterdrops in a hot pot.wav',
        'Dry leaves falling on cement floor.wav', 'Wood chips.wav'],
       dtype='<U133'),
 'captions': array([['a muddled noise of broken channel of the tv',
         'a television blares the rhythm of a static tv ',
         'loud television static dips in and out of focus',
         'the loud buzz of static constantly changes pitch and volume ',
         'heavy static and the beginnings of a signal on a transistor radio'],
        ['a person is turning a map over and over ',
         'a person is very carefully rapping a gift for someone else ',
         'a person is very carefully wrapping a gift for someone else ',
         'he sighed as he turned the pages of the book stopping to scan the information ',
         'papers are being turned stopped then turned again and someone is breathing '],
        ['several 

In [35]:
a1, s1 = torchaudio.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]))
a1_trans = torchaudio.functional.resample(a1, orig_freq=44100, new_freq = 32000)
a11 = a1_trans.reshape(-1).numpy()
a2, s2 = librosa.load(os.path.join('./data/Clotho/waveforms/train/', wav_path[1]), sr=32000)
a2, audio_length2 = pad_or_truncate(a2, max_audio_length)

In [107]:
torch.Tensor(a2.reshape(1,-1))

tensor([[0.0004, 0.0007, 0.0007,  ..., 0.0000, 0.0000, 0.0000]])

In [114]:
a3 = torch.Tensor(a2.reshape(1,-1))
a3_compose = compose_transform(a3)
a3_transform_numpy = a3_compose.numpy()[0]
a3_transform_numpy

array([-0.03096265, -0.01708653, -0.02846347, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [38]:
torchaudio.functional.resample(a1, orig_freq=44100, new_freq = 32000).reshape(-1)

tensor([ 0.0004,  0.0007,  0.0007,  ..., -0.0023, -0.0022, -0.0018])

In [66]:
compose_transform = ComposeTransform([
    RandomClip(sample_rate=sampling_rate, clip_length=64000),
    RandomSpeedChange(sampling_rate),
    RandomBackgroundNoise(sampling_rate, './data/musan/noise')])


In [68]:
from tools.audio_augment import RandomClip, RandomSpeedChange, RandomBackgroundNoise, ComposeTransform
#reshaping_a1 = torch.Tensor(a1.reshape(1,-1))
transformed_audio = compose_transform(torch.Tensor(a1.reshape(1,-1)))
transformed_audio_numpy = transformed_audio.numpy()[0] #torchaudio.load() returns a 2-dimensional tensor, select the first channel.

transformed_audio_numpy

array([-0.00850021, -0.00629861, -0.00925778, ...,  0.        ,
        0.        ,  0.        ], dtype=float32)

In [76]:
audio_name = 'fillerup.wav'
new_filename = audio_name[:-4] + '_aug.wav'
output_augment_path = os.path.join(data_path, 'waveforms/train_augment_only')
output_path = os.path.join(output_augment_path, new_filename)

librosa.output.write_wav(output_path, transformed_audio_numpy, sr = 32000)

AttributeError: module 'librosa' has no attribute 'output'

In [80]:
import soundfile as sf
sf.write(output_path, transformed_audio_numpy, 32000, format='WAV')

In [151]:
import os
import torch
import librosa
from tqdm import tqdm
import soundfile as sf
from tools.audio_augment import RandomClip, RandomSpeedChange, RandomBackgroundNoise


#path 지정해주기
data_path = '/home/clim-lab/바탕화면/main_drive/jiwon/retrieval/data/Clotho'
csv_path = os.path.join(data_path, 'csv_files/')

train_csv_path = os.path.join(csv_path, 'train.csv')
train_audio_dir = os.path.join(data_path, 'waveforms/train/')
# train_hdf5_path = os.path.join(data_path, 'hdf5s/train/train.h5')

train_meta_dict = load_metadata(dataset, train_csv_path)
dataset='Clotho'


compose_transform = ComposeTransform([
    RandomClip(sample_rate=sampling_rate, clip_length=64000),
    RandomSpeedChange(sampling_rate),
    RandomBackgroundNoise(sampling_rate, './data/musan/noise')])

    

def augment_raw_audio(dataset, train_meta_dict):
    '''
    making raw audio augmentation using randomclip, randomnoise, randomspeedchange
    file saved in './data/Clotho/waveforms/
    '''

    output_augment_path = os.path.join(data_path, 'waveforms/train_augment_only22')
    audio_nums = len(train_meta_dict['audio_name'])

    for i in tqdm(range(audio_nums)):
        audio_name = train_meta_dict['audio_name'][i]

        #using torchaudio.load
        audio, sr = librosa.load(train_audio_dir + audio_name, sr=sampling_rate, mono=True)
        #audio, audio_length = pad_or_truncate(audio, max_audio_length)
                                 
        audio_trans = torch.Tensor(audio.reshape(1,-1))
        
        new_filename = audio_name[:-4] + '_aug.wav'
        output_path = os.path.join(output_augment_path, new_filename)


        #대충 흐름은 이렇게 간다 
        #audio augmentation 3가지 한방에 적용시키기
        
        transformed_audio = compose_transform(audio_trans)
        transformed_audio_numpy = transformed_audio.numpy()[0] #torchaudio.load() returns a 2-dimensional tensor, select the first channel.


        sf.write(output_path, transformed_audio_numpy, 32000, format='WAV')


augment_raw_audio(dataset, train_meta_dict)

100%|███████████████████████████████████████████████████████████████████████████| 3839/3839 [1:45:14<00:00,  1.64s/it]


----
### augment된 오디오에 대해서 캡션 csv 파일 생성하기

In [170]:
import pandas as pd
import os
train_df = pd.read_csv(train_csv_path)
train_aug_df = train_df.copy()

In [171]:
b1, e1 = os.path.splitext(train_df['file_name'][171])
print(train_df['file_name'][171])
print(b1)
print(e1)

20061208.waves.02.wav
20061208.waves.02
.wav


In [172]:
def add_aug_suffix(file_name):
    base_name, ext = os.path.splitext(file_name)
    if ext.lower() == ".wav":
        return base_name + "_aug.wav"
    return file_name

In [174]:
train_aug_df["file_name"] = train_aug_df["file_name"].apply(add_aug_suffix)
train_df_all = pd.concat([train_df, train_aug_df], ignore_index=True)
display(train_df_all.sort_values(by='file_name', ascending=True))

,file_name,caption_1,caption_2,caption_3,caption_4,caption_5
3614,Ambience Birds.wav,A wild assortment of birds are chirping and ca...,Several different types of bird are tweeting a...,"Birds tweeting and chirping happily, engine in...",An assortment of wild birds are chirping and ...,Birds are chirping and making loud bird noises.
7453,Ambience Birds_aug.wav,A wild assortment of birds are chirping and ca...,Several different types of bird are tweeting a...,"Birds tweeting and chirping happily, engine in...",An assortment of wild birds are chirping and ...,Birds are chirping and making loud bird noises.
3402,typical neighborhood in Porto.wav,people talking to and over each other in a bus...,"A child and a man are speaking to each other, ...",People in a busy market talk to and about each...,"The man yells, as the child and the man are sp...",A couple of people are talking back and forth.
7241,typical neighborhood in Porto_aug.wav,people talking to and over each other in a bus...,"A child and a man are speaking to each other, ...",People in a busy market talk to and about each...,"The man yells, as the child and the man are sp...",A couple of people are talking back and forth.
3285,&quot;BCat Bites a Bit&quot; .wav,The man is repeating words over and over with ...,A person with a clear voice is repeating some ...,A woman is repeating what she says at normal v...,Somebody speaks out loud and endlessly repeat...,A young person repeats his words while trying ...
...,...,...,...,...,...,...
7573,young goat bleating in a stable_aug.wav,A goat is bleating loudly while people are spe...,A goat bleats noisily while people in the back...,"A goat bleats loudly as time goes by, while pe...",a goat bleats loudly as time goes on while peo...,Humans chatter among themselves as a lamb is b...
632,zipper.wav,A long zipper is tested by being repeatedly pu...,A zipper is being opened and closed quickly.,Someone is rapidly zipping and unzipping a zip...,a really long zipper is being pulled and teste...,a zipper being opened and closed at varying sp...
4471,zipper_aug.wav,A long zipper is tested by being repeatedly pu...,A zipper is being opened and closed quickly.,Someone is rapidly zipping and unzipping a zip...,a really long zipper is being pulled and teste...,a zipper being opened and closed at varying sp...
709,zippers01.wav,A jacket or bag is being zipped and unzipped.,A long zipper is clasped frequently back and f...,A zipper to a suitcase is being opened and clo...,Long zipper being zipped back and forth multip...,The zipper is opened and closed at a slow pace.


In [175]:
# train_aug_df.to_csv('./data/Clotho/csv_files/train_augment_only.csv', index=False)
# train_df_all.to_csv('./data/Clotho/csv_files/train_augment.csv', index=False)

In [176]:
display(train_df['caption_1'].nunique())
display(train_df_all['caption_1'].nunique())

3836

3836

----
### train augment 된 데이터와 기존 train 데이터 폴더들 합치기


In [153]:
import os
import shutil
from tqdm import tqdm

def merge_folders(source_folder1, source_folder2, merge_folder):
    '''
    두 개의 폴더 내용물을 합쳐서 새로운 폴더를 만듦
    '''
    # 폴더가 존재하지 않으면 생성
    if not os.path.exists(merge_folder):
        os.makedirs(merge_folder)

    # source_folder1의 파일 -> merge_folder로 복사
    for filename in tqdm(os.listdir(source_folder1), desc="Copying files from train waveforms", unit="file"):
        src = os.path.join(source_folder1, filename)
        dst = os.path.join(merge_folder, filename)
        shutil.copy(src, dst)

    # source_folder2의 파일 -> merge_folder로 복사
    for filename in tqdm(os.listdir(source_folder2), desc="Copying files from augmented train waveform", unit="file"):
        src = os.path.join(source_folder2, filename)
        dst = os.path.join(merge_folder, filename)
        shutil.copy(src, dst)

#경로 지정
source_folder1 = './data/Clotho/waveforms/train' 
source_folder2 = './data/Clotho/waveforms/train_augment_only' 
merge_folder = './data/Clotho/waveforms/train_augment'  # 전체 합칠 폴더 경로

merge_folders(source_folder1, source_folder2, merge_folder)


Copying files from augmented train waveform: 100%|██████████████████████████████| 3839/3839 [02:08<00:00, 29.92file/s]


------

### csv 파일에 있는 file_name 과 train_augment 폴더에 있는 파일들 비교하기

In [185]:
import os
import pandas as pd
import logging

def check_missing_files(csv_file_path, augment_folder_path):
    #logger사용해서 찍기
    logger = logging.getLogger("missing_files_logger")
    logger.setLevel(logging.INFO)
    
    # Create a console handler
    console_handler = logging.StreamHandler()
    console_handler.setLevel(logging.INFO)
    formatter = logging.Formatter('%(message)s')
    console_handler.setFormatter(formatter)

    # Add the console handler to the logger
    logger.addHandler(console_handler)
    
    
    #csv 파일 위치와 file_name 불러오기
    data = pd.read_csv(csv_file_path)
    csv_filenames = data['file_name'].tolist()

    #augment한 waveform 파일 이름들 불러오기
    augment_filenames = os.listdir(augment_folder_path)

    # Find missing files in 'train_augment' folder
    missing_files = [filename for filename in csv_filenames if filename not in augment_filenames]
    
    # Log the missing files
    logger.info("Missing files in 'train_augment' folder:")
    if not missing_files:
        logger.info("No missing files")
    else:
        for filename in missing_files:
            logger.info(filename)

    return missing_files

csv_file_path = './data/Clotho/csv_files/train_augment.csv'
augment_folder_path = './data/Clotho/waveforms/train_augment' 

check_missing_files(csv_file_path, augment_folder_path)

Missing files in 'train_augment' folder:
No missing files


[]

----
### h5 train file 만들기

In [142]:
from tools.dataset import load_metadata, pad_or_truncate, _create_vocabulary, _sentence_process
from tools.dataset import pack_dataset_to_hdf5

In [144]:
from tools.dataset import pack_augment_dataset_to_hdf5

ImportError: cannot import name 'pack_augment_dataset_to_hdf5' from 'tools.dataset' (/home/clim-lab/바탕화면/main_drive/jiwon/retrieval/tools/dataset.py)

In [149]:
def pack_augment_dataset_to_hdf5(dataset):
    """

    Args:
        dataset: 'AudioCaps', 'Clotho'

    Returns:

    """

    splits = ['train_augment', 'val', 'test']
    sampling_rate = 32000
    all_captions = [] 
    #csv file에 있는 caption들을 list로 한번에 저장 ex) train data -> 3839*5 = 19195개 caption 저장
    
    #if dataset == 'AudioCaps':
    #    audio_duration = 10
    if dataset == 'Clotho':
        audio_duration = 30
        
    else:
        raise NotImplementedError(f'No dataset named: {dataset}')

    max_audio_length = audio_duration * sampling_rate # 30 * 32000
    
    
    for split in splits:
        csv_path = 'data/{}/csv_files/{}.csv'.format(dataset, split)
        audio_dir = 'data/{}/waveforms/{}/'.format(dataset, split)
        
        if split=='train_augment':
            hdf5_path = 'data/{}/hdf5s/{}/'.format(dataset, split)
              
        else:
            hdf5_path = 'data/{}/hdf5s/{}_augment/'.format(dataset, split)

            
        # make dir for hdf5
        Path(hdf5_path).mkdir(parents=True, exist_ok=True)

        meta_dict = load_metadata(dataset, csv_path)
        # meta_dict: {'audio_names': [], 'captions': []}

        audio_nums = len(meta_dict['audio_name'])

        if split == 'train_augment':
            # store all captions in training set into a list
            if dataset == 'Clotho':
                for caps in meta_dict['captions']:
                    for cap in caps:
                        all_captions.append(cap)
            else:
                all_captions.extend(meta_dict['captions'])

        start_time = time.time()
        

        with h5py.File(hdf5_path+'{}.h5'.format(split), 'w') as hf:

            hf.create_dataset('audio_name', shape=(audio_nums,), dtype=h5py.special_dtype(vlen=str))
            hf.create_dataset('audio_length', shape=(audio_nums,), dtype=np.uint32)
            hf.create_dataset('waveform', shape=(audio_nums, max_audio_length), dtype=np.float32)

            if split == 'train' and dataset == 'AudioCaps':
                hf.create_dataset('caption', shape=(audio_nums,), dtype=h5py.special_dtype(vlen=str))
            
            else: #'clotho dataset'
                hf.create_dataset('caption', shape=(audio_nums, 5), dtype=h5py.special_dtype(vlen=str))

            for i in tqdm(range(audio_nums)):
                audio_name = meta_dict['audio_name'][i]

                audio, _ = librosa.load(audio_dir + audio_name, sr=sampling_rate, mono=True)
                audio, audio_length = pad_or_truncate(audio, max_audio_length) 
                #이미 augment할때 잘라버렸는데... 
                

                hf['audio_name'][i] = audio_name.encode()
                hf['audio_length'][i] = audio_length
                hf['waveform'][i] = audio
                hf['caption'][i] = meta_dict['captions'][i]

        logger.info(f'Packed {split} set to {hdf5_path} using {time.time() - start_time} s.')
    
    words_list, words_freq = _create_vocabulary(all_captions)
    # 총 4368개(augment전)
    
    logger.info(f'Creating vocabulary: {len(words_list)} tokens!')
    write_pickle_file(words_list, 'data/{}/pickles/words_list_augment.p'.format(dataset))
    
pack_augment_dataset_to_hdf5('Clotho')

 52%|████████████████████████████████████████▏                                    | 4010/7678 [33:02<30:13,  2.02it/s]


FileNotFoundError: [Errno 2] No such file or directory: 'data/Clotho/waveforms/train_augment/20061208_aug.waves.02_aug.wav'

In [186]:
def pack_augment_dataset_to_hdf5(dataset):
    """

    Args:
        dataset: 'AudioCaps', 'Clotho'

    Returns:

    """

    splits = ['train_augment', 'val', 'test']
    sampling_rate = 32000
    all_captions = [] 
    #csv file에 있는 caption들을 list로 한번에 저장 ex) train data -> 3839*5 = 19195개 caption 저장
    
    #if dataset == 'AudioCaps':
    #    audio_duration = 10
    if dataset == 'Clotho':
        audio_duration = 30
        
    else:
        raise NotImplementedError(f'No dataset named: {dataset}')

    max_audio_length = audio_duration * sampling_rate # 30 * 32000
    
    for split in splits:
        csv_path = 'data/{}/csv_files/{}.csv'.format(dataset, split)
        audio_dir = 'data/{}/waveforms/{}/'.format(dataset, split)
        
        if split=='train_augment':
            hdf5_path = 'data/{}/hdf5s/{}/'.format(dataset, split)
              
        else:
            hdf5_path = 'data/{}/hdf5s/{}_augment/'.format(dataset, split)

            
        # make dir for hdf5
        Path(hdf5_path).mkdir(parents=True, exist_ok=True)

        meta_dict = load_metadata(dataset, csv_path)
        # meta_dict: {'audio_names': [], 'captions': []}

        audio_nums = len(meta_dict['audio_name'])

        if split == 'train_augment':
            # store all captions in training set into a list
            if dataset == 'Clotho':
                for caps in meta_dict['captions']:
                    for cap in caps:
                        all_captions.append(cap)
            else:
                all_captions.extend(meta_dict['captions'])

        start_time = time.time()
        
        try:
            with h5py.File(hdf5_path+'{}.h5'.format(split), 'w') as hf:

                hf.create_dataset('audio_name', shape=(audio_nums,), dtype=h5py.special_dtype(vlen=str))
                hf.create_dataset('audio_length', shape=(audio_nums,), dtype=np.uint32)
                hf.create_dataset('waveform', shape=(audio_nums, max_audio_length), dtype=np.float32)

                if split == 'train' and dataset == 'AudioCaps':
                    hf.create_dataset('caption', shape=(audio_nums,), dtype=h5py.special_dtype(vlen=str))

                else: #'clotho dataset'
                    hf.create_dataset('caption', shape=(audio_nums, 5), dtype=h5py.special_dtype(vlen=str))

                for i in tqdm(range(audio_nums)):
                    audio_name = meta_dict['audio_name'][i]

                    audio, _ = librosa.load(audio_dir + audio_name, sr=sampling_rate, mono=True)
                    audio, audio_length = pad_or_truncate(audio, max_audio_length) 
                    


                    hf['audio_name'][i] = audio_name.encode()
                    hf['audio_length'][i] = audio_length
                    hf['waveform'][i] = audio
                    hf['caption'][i] = meta_dict['captions'][i]
        except FileNotFoundError as F:
            print("file not found", F)
        except ValueError:
            print("Invalid value.")
        except Exception as e:
            print(f"Error: {e}")
            

        logger.info(f'Packed {split} set to {hdf5_path} using {time.time() - start_time} s.')
    
    words_list, words_freq = _create_vocabulary(all_captions)
    # 총 4368개(augment전)
    
    logger.info(f'Creating vocabulary: {len(words_list)} tokens!')
    write_pickle_file(words_list, 'data/{}/pickles/words_list_augment.p'.format(dataset))


- 새로만든 폴더 train_augment폴더에 다시 합쳐서 만들기 -> **DONE** 
- csv 파일 만들때 suffix나 정규식 사용해서 확장자일 경우에만 분리하도록 코드 다시짜기 -> **DONE** 
- 오디오 파일 os.listdir 해서 여기에 있는 오디오 이름들이 csv_file에도 있는지 확인코드 짜기 -> **DONE** 
- data_augment.py 실행시켜서 h5py 파일 생성하기 -> 돌리기 전에 코드 세세히 보고 수정하기 -> **DONE** 
- h5py파일 만들때 try except 구문 넣어서 예외처리생겼을 떄는 로그만 찍고 넘어가기 -> **DONE** 

hdf5파일 만들어지면 폴더 잘 생성되었는지, audio length 제대로 됐는지 확인하기 -> **DONE**      
-> 기존보다 더 짧게 됐네....

In [188]:
data_path = '/home/clim-lab/바탕화면/main_drive/jiwon/retrieval/data/Clotho'
train_h5_path = os.path.join(data_path, 'hdf5s/train_augment/train_augment.h5')
val_h5_path = os.path.join(data_path, 'hdf5s/val/val.h5')
test_h5_path = os.path.join(data_path, 'hdf5s/test/test.h5')

In [209]:
with h5py.File(train_h5_path, 'r') as hf:
    train_audio_keys = [audio_name.decode() for audio_name in hf['audio_name'][:]]
    # audio_names: [str] 
    train_audio_lengths = [length for length in hf['audio_length'][:]]
                # [cap_1, cap_2, ..., cap_5]
    train_captions = hf['caption'][:]
    
with h5py.File(val_h5_path, 'r') as hf:
    val_audio_keys = [audio_name.decode() for audio_name in hf['audio_name'][:]]
    # audio_names: [str] 
    val_audio_lengths = [length for length in hf['audio_length'][:]]
    val_captions = hf['caption'][:]
    
with h5py.File(test_h5_path, 'r') as hf:
    test_audio_keys = [audio_name.decode() for audio_name in hf['audio_name'][:]]
    # audio_names: [str] 
    test_audio_lengths = [length for length in hf['audio_length'][:]]
    test_captions = hf['caption'][:]
    


In [211]:
display(len(train_audio_keys))
display(len(val_audio_keys))
display(len(test_audio_keys))

print("길이 비교")
display(sum(train_audio_lengths)/len(train_audio_lengths))
display(sum(val_audio_lengths)/len(val_audio_lengths))
display(sum(test_audio_lengths)/len(test_audio_lengths))

7678

1045

1045

길이 비교


645889.61747851

723793.4411483253

717054.9693779905

In [212]:
train_captions[:3]

array([[b'a muddled noise of broken channel of the tv',
        b'a television blares the rhythm of a static tv ',
        b'loud television static dips in and out of focus',
        b'the loud buzz of static constantly changes pitch and volume ',
        b'heavy static and the beginnings of a signal on a transistor radio'],
       [b'a person is turning a map over and over ',
        b'a person is very carefully rapping a gift for someone else ',
        b'a person is very carefully wrapping a gift for someone else ',
        b'he sighed as he turned the pages of the book stopping to scan the information ',
        b'papers are being turned stopped then turned again and someone is breathing '],
       [b'several barnyard animals mooing in a barn while it rains outside ',
        b'the vocalization of several whales along with the clicking of large numbers of shrimp reverberated below in the water ',
        b'underwater large numbers of shrimp clicking and several whales vocalizing ',